In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [2]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df.shape

(34886, 8)

# Preprocessing

In [3]:
df = df[df['Genre'] != 'unknown']
df.shape

(28803, 8)

In [4]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...
12,1906,Kathleen Mavourneen,American,Edwin S. Porter,NaN,short film,https://en.wikipedia.org/wiki/Kathleen_Mavourn...,Irish villager Kathleen is a tenant of Captain...


In [5]:
df['Genre'].nunique()

2264

In [6]:
# 1.1. dictionary-based conversion readacting Wikipedia's Film genre page:

conversion_dict = {
    "action": ["disaster", "martial arts", "spy", "superhero", "wuxia","action","masala","espionage","arts"],
    "adventure": ["pirate", "swashbuckler", "samurai"],
    "animation": ["cgi", "cutout", "live-action animated film", "stop motion", "animated", "computer-animated", "anime"],
    "comedy": ["buddy", "mockumentary", "parody", "slapstick"],
    "drama": ["docudrama", "melodrama", "biodrama", "bio-drama"],
    "historical": ["history", "historic", "alternate history", "period", "period piece", "biopic", "bio-pic", "biographical"],
    "horror": ["ghost", "monster", "vampire", "werewolf", "slash", "splatter", "zombie", "j-horror","supernatural"],
    "science fiction": ["dystopian", "dystopia", "post-apocalyptic", "steampunk", "tech noir", "utopian", "science-fiction", "scifi", "sci-fi", "space", "tokusatsu","fiction"],
    "thriller": ["mystery", "detective", "crime","suspense"],
    "musical": ["operetta"],
    "romance": ["love","romantic"],
    "western": ["cowboy"],
    "documentary": ["pseudo-documentary"],
    "fantasy":[],
    "sport":["sports","races","dance","biker"],
    "war":['ii','i'],
    "erotic":['ero','adult','erotic','sexploitation'],
    "social":['socio','costume']
}

In [7]:
import re

#preprocessing function helper for genres reduction
def genres_preprocessing(genres_dict, genre):
    stop = ['film','short']
    for w in stop:
        if w in genre:
            genre = genre.replace(w,'').strip()
        
    
    splitted = re.split("[,/]", genre)
    if len(splitted) != 1:
        genre = splitted[0]
        
    splitted = re.split("[-—–]", genre)
    if any(item in genres_dict.keys() for item in splitted):
    #if splitted[0] in genres_dict.keys():
        genre = splitted[0]
    
    splitted = re.split(" ", genre.rstrip())
    if len(splitted) != 1:
        genre = splitted[-1]
        
    for key in genres_dict.keys():
        if genre.rstrip() in genres_dict[key]:
            genre = key
        
    return genre.rstrip()

In [8]:
df['Genre2'] = df['Genre']
df['Genre2'] = df['Genre2'].apply(lambda x: genres_preprocessing(conversion_dict, x))

In [9]:
counts = df['Genre2'].value_counts()
to_remove = counts[counts < 50].index

df = df[~df.Genre2.isin(to_remove)]

drop_id = df[df.Plot.apply(lambda x: len(x) < 25)].index
df.drop(drop_id, inplace=True)

df = df[df['Genre2'] != '']

In [10]:
#g = df.groupby("Genre2")
#new_df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

# Scraping

In [11]:
vc = df['Genre2'].value_counts()
genres_to_fetch = vc[vc < 1000].index.tolist()
genres_to_fetch

['western',
 'animation',
 'science fiction',
 'adventure',
 'musical',
 'war',
 'noir',
 'family',
 'fantasy',
 'historical',
 'biography',
 'social',
 'documentary',
 'serial',
 'sport']

In [12]:
import requests
from bs4 import BeautifulSoup

def get_links(src, stw="/", genre=""):
    links = []
    soup = BeautifulSoup(src)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and href.startswith(stw) and genre.replace(" ","_") in href.lower():
            links.append(href)
            
    return links

def request_links(base, in_links):
    out_links = []
    if type(in_links) == list:
        response = [ requests.get(base+link).text for link in set(in_links) ]
        for res in response:
            out_links.extend(get_links(res))

    else:
        response = requests.get(base+in_links).text
        out_links.extend(get_links(response))
        
    return out_links

In [13]:
def fetching_plot(link):
    film_dict = dict()
    html = requests.get(link).text
    soup = BeautifulSoup(html)
    if soup.find("span", {"id":"Plot"}):
        title = soup.find("h1", {"id":"firstHeading"}).text
        print(title)
        plot_span = soup.find("span", {"id":"Plot"})
        obj = plot_span.find_parent()
        plot = ""
        while True:
            obj = obj.next_sibling
            if obj.name not in ['p','h2']:
                continue
            elif obj.name == 'p':
                plot += obj.text.strip('\n')
            elif obj.name == 'h2':
                break
                
        return title, plot

In [14]:
import time
from concurrent.futures import ThreadPoolExecutor
import threading
from functools import partial

films_dict = dict()

def plot_scraper(genre, link):
    f_link = "https://en.wikipedia.org"+link
    title, plot = fetching_plot(f_link) or (None, None)
    films_dict[title] = plot
    time.sleep(1)
    
        
def set_up_threads(links, genre):
    with ThreadPoolExecutor(max_workers=6) as executor:
        return executor.map(partial(plot_scraper,genre),
                            links,
                            timeout = 60)    
            

In [15]:
base = "https://en.wikipedia.org"
genres_link = "https://en.wikipedia.org/wiki/Template:Films_by_genre_sidebar"

main_res = requests.get(genres_link).text

In [ ]:
for genre in genres_to_fetch:
    genre_links = get_links(main_res, "/wiki/List", genre)
    for link in genre_links:
        genre_sublinks = request_links(base, link)
        if "Lists" in link:
            final_genre_sublinks = []
            print(genre.upper())
            for fl in genre_sublinks:
                genre_to_match = genre.title() if genre == 'western' else genre.replace(" ", "_")
                genre_to_match = "List_of_"+genre_to_match+"_films"
                if genre_to_match in fl:
                    real_fl = request_links(base, fl)
                    final_genre_sublinks.extend(real_fl)
            set_up_threads(final_genre_sublinks, genre)
        else:
            print(genre.upper())
            set_up_threads(genre_sublinks, genre)
            

WESTERN
The Great Train Robbery (1903 film)
The Comancheros (film)
Kidnapping by Indians
The Great Train Robbery (1903 film)
Kit Carson (1903 film)
The Pioneers (1903 film)
Bushranging in North Queensland
The Hold-Up of the Leadville Stage
The Little Train Robbery
The Train Wreckers
From Leadville to Aspen: A Hold-Up in the Rockies
The Story of the Kelly Gang
The Fight for Freedom
The Cowboy Millionaire (1909 film)
The Last Drop of Water
Algie the Miner
Geronimo's Last Raid
The Girl and Her Trust
The Battle at Elderbush Gulch
Her Grave Mistake
A Miner's Romance
A Ranch Romance
Rose of the Rancho
The Squaw Man (1914 film)
The Tragedy of Whispering Creek
The Desert Breed
The Girl of the Golden West (1915 film)
Accusing Evidence
The Committee on Credentials
Hell's Hinges
Liberty (serial)
Love's Lariat
The Return of Draw Egan
A 44-Calibre Mystery
The Bad Man of Cheyenne
The Empty Gun
The Fighting Trail
A Marked Man
The Narrow Trail
The Silent Man (film)
Six-Shooter Justice
Straight Shootin

In [ ]:
films_dict

In [78]:
def fetching_plot(link):
    film_dict = dict()
    html = requests.get(link).text
    soup = BeautifulSoup(html)
    
    # plot fetching
    if soup.find("span", {"id":"Plot"}):
        title = soup.find("h1", {"id":"firstHeading"}).text
        print(title)
        plot_span = soup.find("span", {"id":"Plot"})
        obj = plot_span.find_parent()
        plot = ""
        while True:
            obj = obj.next_sibling
            if obj.name not in ['p','h2']:
                continue
            elif obj.name == 'p':
                plot += obj.text.strip('\n')
            elif obj.name == 'h2':
                break
                
    if soup.find("span", {"id":"Cast"}):
        cast_span = soup.find("span", {"id": "Cast"})
        obj = cast_span.find_parent()
        cast = ""
        while True:
            obj = obj.next_sibling
            if type(obj) != None:
                if obj.name not in ['ul', 'li']:
                    continue
                elif obj.name == 'ul':
                    print("test"+obj.name)
                    cast += obj.text.strip('li')
                elif obj.name == 'h2':
                    break
            
    return title, plot, cast

In [79]:
import time
from concurrent.futures import ThreadPoolExecutor
import threading
from functools import partial

films_dict = dict()

def plot_scraper(genre, link):
    f_link = "https://en.wikipedia.org"+link
    title, plot, cast = fetching_plot(f_link) or (None, None, None)
    films_dict[title] = plot, genre, cast
    time.sleep(1)
    
        
def set_up_threads(links, genre):
    with ThreadPoolExecutor(max_workers=6) as executor:
        return executor.map(partial(plot_scraper,genre),
                            links,
                            timeout = 60) 

In [80]:
test_link = "/wiki/Good_Morning_(1959_film)"
plot_scraper("comedy", test_link)

Good Morning (1959 film)
testul


AttributeError: 'NoneType' object has no attribute 'name'

In [ ]:
films_dict

# Classification

### SVM

In [10]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre2'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [11]:
len(X_train),len(y_train),len(X_test),len(y_test)

(21029, 21029, 7010, 7010)

In [12]:
import nltk

nlp = spacy.load('en_core_web_sm')

doc_counter = 0
def reset_counter():
  global doc_counter
  doc_counter = 0

def increase_counter():
  global doc_counter
  doc_counter += 1
  if doc_counter % 100 == 0:
    print(doc_counter)

def spacy_nlp_tokenizer(text):
    increase_counter()
        
    # we use spacy for main nlp tasks
    doc = nlp(text)
    # lemmatized tokens, skipping stopwords
    lemmas = ['LEMMA_'+token.lemma_ for token in doc if not token.is_stop]
    # entity_types
    entity_types = ['NER_'+token.ent_type_ for token in doc if token.ent_type_]

    # in case an entity linker is available, we can use it do put actual entities as
    # features, e.g. Queen Elizabeth, Elizabeth II, Her Majesty -> KB2912
    # see https://spacy.io/usage/training#entity-linker
    # entities = ['ENT_'+token.ent_kb_id_ for token in doc if token.ent_kb_id_]

    # we use a simple nltk function to create ngrams
    lemma_bigrams = ['BI_'+p1+'_'+p2 for p1,p2 in nltk.ngrams(lemmas,2)]
    lemma_trigrams = ['TRI_'+p1+'_'+p2+'_'+p3 for p1,p2,p3 in nltk.ngrams(lemmas,3)]

    all_tokens = list()
    all_tokens.extend(lemmas)
    all_tokens.extend(lemma_bigrams)
    all_tokens.extend(lemma_trigrams)
    all_tokens.extend(entity_types)
    return all_tokens

In [13]:
vect = CountVectorizer(analyzer=spacy_nlp_tokenizer, min_df=5)

X_train_tok = vect.fit_transform(X_train)

X_test_tok = vect.transform(X_test)

100
200
300
400
500
600
700
800
900


KeyboardInterrupt: 

In [ ]:
X_train_tok.shape, X_test_tok.shape

In [ ]:
import pickle

with open('x_train_tok.pkl','wb') as outfile:
    pickle.dump(X_train_tok,outfile)
with open('x_test_tok.pkl','wb') as outfile:
    pickle.dump(X_test_tok,outfile)

In [12]:
import pickle

infile = open('x_train_tok.pkl','rb')
X_train_tok = pickle.load(infile)
infile.close()

infile = open('x_test_tok.pkl','rb')
X_test_tok = pickle.load(infile)
infile.close()

In [13]:
X_train_tok.shape, X_test_tok.shape

((21029, 6806178), (7010, 6806178))

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'sel__k': [1000, 2000, 5000, 10000], 'learner__C': [0.01, 0.1, 1, 10]},
 ]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])


n_jobs = 2

opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=3).fit(X_train_tok,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [15]:
opt_predictions = opt_search.best_estimator_.predict(X_test_tok)

correct = 0
for prediction,true_label in zip(opt_predictions, y_test):
    if prediction==true_label:
        correct += 1
print(correct/len(opt_predictions))

0.2905848787446505


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
print('Classification report:')
print(classification_report(y_test, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
                 precision    recall  f1-score   support

         action       0.00      0.00      0.00       516
      adventure       0.00      0.00      0.00       166
      animation       0.00      0.00      0.00       205
           arts       0.00      0.00      0.00        18
      biography       0.00      0.00      0.00        55
         comedy       0.24      0.02      0.04      1636
    documentary       0.00      0.00      0.00        23
          drama       0.29      0.98      0.45      2041
         family       0.00      0.00      0.00        81
        fantasy       0.00      0.00      0.00        78
        fiction       0.00      0.00      0.00       117
     historical       0.00      0.00      0.00        63
         horror       0.00      0.00      0.00       342
        musical       0.00      0.00      0.00       139
           noir       0.00      0.00      0.00        86
        romance       0.00      0.00      0.00       335
science

C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
